In [1]:
#!/usr/bin/python3
import os
import numpy as np
import string
import pandas as pd
from pandas import DataFrame
import random
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import math
from collections import defaultdict
from BTrees.OOBTree import OOBTree
import datetime as dt

ImportError: No module named BTrees.OOBTree

In [ ]:
#This routine returns all the paths from the node v
def DFS(G,v,seen=None,path=None):
    if seen is None: seen = []
    if path is None: path = [v]

    seen.append(v)

    paths = []
    for t in G[v]:
        if t not in seen:
            t_path = path + [t]
            paths.append(tuple(t_path))
            paths.extend(DFS(G, t, seen[:], t_path))
    return paths

#Given a set of paths this routine returns the maximum length chain
def give_a_max_path(all_paths):
        
    
    max_len   = max(len(p) for p in all_paths)
    
    max_paths = [p for p in all_paths if len(p) == max_len]
        
    num_max_paths = len(max_paths)
        
    chosen_path = max_paths[np.random.randint(num_max_paths)]
    
    return chosen_path, max_len

def union(list1,list2):
    return list(set(list1) | set(list2))

In [11]:
# Defining the tree dictionary for each node
# Each block tree is a dictionary
# Each node is in nodes[] list

num_nodes = 500

max_blocks = 50

D_REGULAR=4

poll_L = 1

special_sync=False

#Chyeck list of lists

In [12]:
delay_vector= np.arange(0.2,1.1,0.1)
delay_vector = [0.1]
print(delay_vector)

num_delays=len(delay_vector)

num_trials = 16

[0.1]


In [13]:
max_len_tracker = np.zeros((num_delays, max_blocks+1, num_trials))

final_len_array=[]
#final_len_array.append(1)
#print(final_len_array)

slow_fraction = 0.0
slow_factor = 1

node_delay = {}
edge_delay = {}

G = nx.random_regular_graph(D_REGULAR,num_nodes)  # d-regular
#print(G.adj)
edge_set = set()
adj_list = G.adj
for i in adj_list:
    for j in adj_list[i]:
        edge_set.add(num_nodes*min(i,j) + max(i,j))
print(len(edge_set))



1000


In [14]:
for d in range(num_delays):
    
    Delay = delay_vector[d]
    D_tilde = Delay*0.01 
    data={} 
    
    for i in range(num_nodes):
        node_delay[i] = Delay + (slow_factor-1)*Delay*np.random.binomial(1,slow_fraction)
    for edge in edge_set:
        n1 = int(edge/num_nodes)
        n2 = edge%num_nodes
        edge_delay[edge] = max(node_delay[n1],node_delay[n2])
    
        
    print("Starting with delay:", Delay)
    
    for trial in range(num_trials):
    
        nodes = num_nodes*[0]

        #Tree variables
        num_in_strings = [str(i) for i in range(max_blocks+1)]

        for i in range(num_nodes):
            nodes[i] = {i:[] for i in num_in_strings[:-1]}

        # nodes[i] is a dictionary of block tree for node i which contains children of the blocks 0,1,....,T-1    

        god_node_children = {i:[] for i in num_in_strings}    
        god_node_parents = {i:"" for i in num_in_strings[:-1]}

        #rechability_matrix= np.zeros((num_nodes,max_blocks+1))
        #rechability_matrix[:,0]=1
        
        #create chosen nodes array
        chosen_node_array = []
        for i in range(max_blocks):
            chosen_node_array.append(np.random.randint(num_nodes))
        
        node_sync_list = {}
        for i in range(num_nodes):
            node_sync_list[i] = []
        
        # update process pretiming for each edges
        t1 = dt.datetime.now()
        event_dict = {}
        for edge in edge_set:
            first_event_bool= True
            n1 = int(edge/num_nodes)
            n2 = edge%num_nodes
            temp_time=0
            while(temp_time<=max_blocks):
                prev_temp_time = temp_time 
                temp_time+=np.random.exponential(edge_delay[edge])
                node_sync_list[n1].append(temp_time)
                node_sync_list[n2].append(temp_time)
                if not first_event_bool:
                    
                    event_dict[prev_temp_time] = event_dict[prev_temp_time][:3]+[temp_time]+event_dict[prev_temp_time][3:]
                    
                event_dict[temp_time]=[temp_time,n1,n2]
                if first_event_bool:
                    first_event_bool=False
                    event_dict[temp_time].append(nodes[0])
                    event_dict[temp_time].append(nodes[0])
            event_dict[temp_time] = event_dict[temp_time][:3]+[None]+event_dict[temp_time][3:]
        t2 = dt.datetime.now()  
        event_btree = OOBTree()
        event_btree.update(event_dict)
        t3 = dt.datetime.now()
        #print("completed building btree")
        #print('Time taken: ', t2-t1)
        #print('Time taken: ', t3-t2)
        
        '''
        #updating for time 0-1 wont do any update but is necessary to fill pregraphs
        event_list = list(event_btree.values(0,1))
        for event in event_list:
            graph1 = nodes[event[1]]
            graph2 = nodes[event[2]]
            pregraph1 = event[4]
            pregraph2 = event[5]
            nodes[event[1]] = {x:union(graph1[x],pregraph2[x]) for x in graph1}
            nodes[event[2]] = {x:union(graph2[x],pregraph1[x]) for x in graph2}
            
            next_time = event[3]
            
            if next_time is not None:
                event_btree[next_time].append(nodes[event[1]])
                event_btree[next_time].append(nodes[event[2]])
        
        t4 = dt.datetime.now()
        print('Time: ', t4-t3)
        '''
        #Block arrival process
        for t in range(1, max_blocks+1):

            chosen_node = np.random.randint(num_nodes)

            # Update block trees for all nodes "n"

            if t==1:
                god_node_children["0"].append(str(t))

                god_node_parents[str(t)] = str(0)
                
                nodes[0]['0'].append('1')
                
                # Updating block trees / applying pairwise sync
                event_list = list(event_btree.values(t-1,t))
                for event in event_list:
                    graph1 = nodes[event[1]]
                    graph2 = nodes[event[2]]
                    pregraph1 = event[4]
                    pregraph2 = event[5]
                    nodes[event[1]] = {x:union(graph1[x],pregraph2[x]) for x in graph1}
                    nodes[event[2]] = {x:union(graph2[x],pregraph1[x]) for x in graph2}

                    next_time = event[3]
                    if next_time is not None:
                        event_btree[next_time].append(nodes[event[1]])
                        event_btree[next_time].append(nodes[event[2]])


            else:

                # Updating block trees / applying pairwise sync
                event_list = list(event_btree.values(t-1,t))
                for event in event_list:
                    graph1 = nodes[event[1]]
                    graph2 = nodes[event[2]]
                    pregraph1 = event[4]
                    pregraph2 = event[5]
                    nodes[event[1]] = {x:union(graph1[x],pregraph2[x]) for x in graph1}
                    nodes[event[2]] = {x:union(graph2[x],pregraph1[x]) for x in graph2}

                    next_time = event[3]
                    if (next_time is not None) and next_time<= max_blocks:
                        event_btree[next_time].append(nodes[event[1]]) #This works because n1 and n2 are in same order
                        event_btree[next_time].append(nodes[event[2]])
                    if next_time == max_blocks + 2:
                        # Propose a new block here if polled
                        past_chosen_node = event[1]
                        past_graph = nodes[past_chosen_node]
                        past_all_paths = DFS(past_graph, "0")
                        if len(past_all_paths)==0:
                            past_chosen_leaf="0"
                        else:
                            past_chosen_path, past_max_len = give_a_max_path(past_all_paths)
                            past_chosen_leaf=past_chosen_path[-1]
                        past_t = event[6]
                        #Attach this node in the Global tree
                        god_node_children[past_chosen_leaf].append(str(past_t))
                        god_node_parents[str(past_t)] = past_chosen_leaf  
                        #Add a block to the proposers block tree
                        nodes[past_chosen_node][past_chosen_leaf].append(str(past_t))
                        
                        # special sync to semi-broadcast blocks can be switched off if needed later
                        if special_sync:
                            immediate_event_bool = {}
                            for nbr in adj_list[past_chosen_node]:
                                immediate_event_bool[nbr] = True

                            for node_event in node_sync_list[past_chosen_node]:

                                time_slippage = 0
                                if node_event <= event[0]:
                                    time_slippage = event_btree[node_event][0]
                                time_slippage = event[0] - time_slippage
                                if node_event > event[0]:
                                    if event_btree[node_event][1] == past_chosen_node:
                                        nbr = event_btree[node_event][2]
                                    else:
                                        nbr = event_btree[node_event][1]
                                    if immediate_event_bool[nbr]:
                                        new_time = node_event+time_slippage

                                        next_node_event=node_event
                                        next_new_time = new_time
                                        while(next_node_event<=max_blocks):
                                            node_event=next_node_event
                                            new_time=next_new_time
                                            node_sync_list[past_chosen_node] = [new_time if x==node_event else x for x in node_sync_list[past_chosen_node]]
                                            print(node_event)
                                            event_btree[new_time] = event_btree[node_event]
                                            del event_btree[node_event]
                                            event_btree[new_time][0] = new_time
                                            next_node_event = event_btree[new_time][3]
                                            next_new_time = next_node_event + time_slippage
                                            event_btree[new_time][3] = next_new_time
                                            node_sync_list[nbr] = [new_time if x==node_event else x for x in node_sync_list[nbr]]
                                            if immediate_event_bool[nbr]:
                                                immediate_event_bool[nbr]=False
                                                event_btree[new_time][4] = nodes[event_btree[new_time][1]]
                                                event_btree[new_time][5] = nodes[event_btree[new_time][2]]
                                break


                                '''    
                                    new_time = node_event+time_slippage
                                    node_sync_list[past_chosen_node] = [new_time if x==node_event else x for x in node_sync_list[past_chosen_node]]
                                    if new_time<= max_blocks:
                                        print(event_btree[node_event])
                                        event_btree[new_time] = event_btree[node_event]
                                        del event_btree[node_event]
                                        event_btree[new_time][0] = new_time
                                        event_btree[new_time][3] = event_btree[new_time][3] + time_slippage
                                        if event_btree[new_time][1] == past_chosen_node:
                                            nbr = event_btree[new_time][2]
                                        else:
                                            nbr = event_btree[new_time][1]
                                        node_sync_list[nbr] = [new_time if x==node_event else x for x in node_sync_list[nbr]]
                                        if immediate_event_bool[nbr]:
                                            print(new_time)
                                            print(event_btree[new_time])
                                            print(nbr)
                                            immediate_event_bool[nbr]=False
                                            event_btree[new_time][4] = nodes[event_btree[new_time][1]]
                                            event_btree[new_time][5] = nodes[event_btree[new_time][2]]
                                '''
                        
                
                #Find the longest path for  chosen node n starting from the genesis block 0,1,,

                graph = nodes[chosen_node]
                
                if poll_L==1:

                    all_paths = DFS(graph, "0")

                    if len(all_paths)==0:

                        chosen_leaf="0"

                    else:

                        chosen_path, max_len = give_a_max_path(all_paths)

                        chosen_leaf=chosen_path[-1]


                    #Attach this node in the Global tree

                    god_node_children[chosen_leaf].append(str(t))

                    god_node_parents[str(t)] = chosen_leaf

                    nodes[chosen_node][chosen_leaf].append(str(t))
                
                else:
                    temp_time_array = np.random.exponential(D_tilde, (poll_L-1))
                    temp_time_array = np.sort(temp_time_array)
                    allowable_nodes = list(range(num_nodes))
                    allowable_nodes.remove(chosen_node)
                    temp_block_array = np.random.choice(allowable_nodes,poll_L-1,replace=False)
                    
                    for i in range(len(temp_time_array)):
                        temp_time = t + temp_time_array[i]
                        event_btree[temp_time] = [temp_time,chosen_node,temp_block_array[i],max_blocks+1,nodes[chosen_node],nodes[temp_block_array[i]],t]
                    event_btree[temp_time][3] = max_blocks+2
                

            _, max_len_tracker[d,t,trial] = give_a_max_path(DFS(god_node_children,"0"))

            
            max_len_tracker[d,t,trial] = max_len_tracker[d,t,trial]-1
            
            
            if t%10 == 0:

                print("Trial {0}: Block {1} arrived : Growth rate of the main chain is {2}".format(trial, t, max_len_tracker[d,t,trial]))
            
            
        t5 = dt.datetime.now()
        print('Net time', t5-t1)
        data["Trial%d"%(trial)]= max_len_tracker[d,:,trial]
        df = pd.DataFrame(data=data)            
        df.to_csv('symmetric/'+'poll1_n%d_T%d_D%.2f.csv'%(num_nodes,max_blocks,Delay))

Starting with delay: 0.1
Trial 0: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 0: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 0: Block 30 arrived : Growth rate of the main chain is 30.0
Trial 0: Block 40 arrived : Growth rate of the main chain is 39.0
Trial 0: Block 50 arrived : Growth rate of the main chain is 49.0
Net time 0:01:07.018947
Trial 1: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 1: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 1: Block 30 arrived : Growth rate of the main chain is 30.0
Trial 1: Block 40 arrived : Growth rate of the main chain is 38.0
Trial 1: Block 50 arrived : Growth rate of the main chain is 48.0
Net time 0:01:06.354802
Trial 2: Block 10 arrived : Growth rate of the main chain is 10.0
Trial 2: Block 20 arrived : Growth rate of the main chain is 20.0
Trial 2: Block 30 arrived : Growth rate of the main chain is 29.0
Trial 2: Block 40 arrived : Growth rate of the main chain is 39.0
Tri

In [195]:
print(event_btree[52.441308787242605])

[52.441308787242605, 35, 39, 54.70847642239885]
